In [3]:
import pandas
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime, date
from collections import defaultdict, OrderedDict

## Read the CSV, parse the 24/7 and late fields into a single field


In [46]:
restaurants = pandas.read_csv('restaurant_database.csv')

restaurants['open'] = ['24/7' if value == 'yes' else 'early' for value in restaurants['24/7?']]
restaurants['open'] = ['late' if (current == 'early' and open_late == 'yes') else current
                       for current, open_late 
                       in zip(restaurants['open'], restaurants['Open late (after 10pm)'])]

restaurants.head()

,Name,Address,Neighborhood,Cuisine type,Coffee?,Alcohol?,Price range,Vegeterian?,Pescaterian?,Take away?,Open late (after 10pm),24/7?,open
0,Small Fish / Japanese restaurant,137 Yeoksam-ro,Gangnam,Japanese,no,yes,"7,500-15,000",no,yes,no,no,no,early
1,Robot Kimbap,Gangnam station,Gangnam,Korean,no,no,"5,000-7,500",yes,yes,yes,no,no,early
2,Din Tai Fung,"1317-31 Seocho-dong, Seocho-gu",Gangnam,Taiwanese,no,yes,"15,000-25,000",no,yes,no,no,no,early
3,Meal Rice Dumplings,"1328-11 Seocho 2(i)-dong, Seocho-gu, Seoul",Gangnam,Korean,no,no,"5,000-7,500",no,no,yes,yes,no,late
4,KBBQ,"810-1, Yeoksam-dong, Gangnam-gu, Seoul",Gangnam,Korean,no,yes,"7,500-15,000",no,no,no,yes,no,late


## Define translation functions

We take the knowledge base, as defined in the CSV (downloaded from Google sheets), and define how to translate it to a Prolog KB. This includes what should be the names for conditions which are boolean (not multivalued), various measured of input sanitization, and other renamings.

In [47]:
def sanitze_to_atom(name, prefix='w'):
    sanitized = name.lower().replace(' ', '_').replace('-', '_').replace('?', '').replace(',', '')
    if sanitized[0].isdigit():
        sanitized = prefix + sanitized        
    return sanitized


BOOLEAN_AFFIRMATIVE_NAME = {
    'coffee': 'serves',
    'alcohol': 'serves',
    'vegeterian': 'serves',
    'pescaterian': 'serves',
    'take_away': 'offers'
}


COLUMNS_TO_IGNORE = (
    'Name', 'Address', 'Open late (after 10pm)', '24/7?'
)


TRUE_VALUES = ('yes', 'y', 'True', 'true', True)


class BooleanTransformer:
    def __init__(self, col_atom):
        self.col_atom = col_atom
        
    def __call__(self, value):
        out_value = BOOLEAN_AFFIRMATIVE_NAME[self.col_atom]
        if not value in TRUE_VALUES:
            out_value = 'not_' + out_value
        
        return '{name}({value})'.format(name=self.col_atom, value=out_value)

    
class MultivaluedTransformer:
    def __init__(self, col_atom):
        self.col_atom = col_atom
        
    def __call__(self, value):
        return '{name}({value})'.format(name=self.col_atom, 
                                        value=sanitze_to_atom(value))

    
def build_transformers(knowledge_base):
    transformers = {}
    multivalued = []
    
    for column in knowledge_base.columns:
        if column in COLUMNS_TO_IGNORE:
            continue
        
        column_atom = sanitze_to_atom(column)
        if column_atom in BOOLEAN_AFFIRMATIVE_NAME:
            transformers[column] = BooleanTransformer(column_atom)
            
        else:
            transformers[column] = MultivaluedTransformer(column_atom)
            multivalued.append(column)
            
    return transformers, multivalued


INFORMATIVE_FIELDS = ('Name', 'Address', 'Neighborhood')
WRITE_PY_TEMPLATE = 'write_py(\'Your restaurant recommendation is {name} at {address} in {neighborhood}\')'            

    
def transform_kb(knowledge_base, sort_columns=None):
    if sort_columns is not None:
        knowledge_base = knowledge_base.sort_values(sort_columns, ascending=False)
        
    transformers, multivalued = build_transformers(knowledge_base)
    output = []
    
    for rest_id, rest_row in knowledge_base.iterrows():
        transformed_values = [transformers[col](rest_row[col]) for col in transformers]
        transformed_values.append(WRITE_PY_TEMPLATE.format(name=rest_row['Name'].replace("'", r"\\'"), 
                                                           address=rest_row['Address'].replace("'", r"\\'"), 
                                                           neighborhood=rest_row['Neighborhood']))
        
        output.append('suggest(rest_{id}) :- {cond}.'.format(id=rest_id, 
                                                             cond=', '.join(transformed_values)))
    
    output.append('')
    for col in transformers:
        col_atom = sanitze_to_atom(col)
        output.append('{col}(X) :- check({col}, X).'.format(col=col_atom))
    
    output.append('')
    output.extend(['multivalued({col}).'.format(col=sanitze_to_atom(col)) for col in multivalued])
    
    return output
    
        

In [48]:
output = transform_kb(restaurants, ['Cuisine type', 'Price range'])
print('\n'.join(output))

suggest(rest_8) :- neighborhood(gangnam), vegeterian(serves), cuisine_type(vietnamese), alcohol(serves), take_away(not_offers), coffee(not_serves), pescaterian(not_serves), open(w24/7), price_range(w7500_15000), write_py('Your restaurant recommendation is EMOI Pho at Gangnam station, Exit 3 in Gangnam').
suggest(rest_9) :- neighborhood(gangnam), vegeterian(serves), cuisine_type(vietnamese), alcohol(serves), take_away(not_offers), coffee(not_serves), pescaterian(not_serves), open(late), price_range(w7500_15000), write_py('Your restaurant recommendation is EMOI Pho at Sinsa-dong Garosu-Gil in Gangnam').
suggest(rest_12) :- neighborhood(gangnam), vegeterian(not_serves), cuisine_type(vietnamese), alcohol(not_serves), take_away(offers), coffee(not_serves), pescaterian(not_serves), open(early), price_range(w5000_7500), write_py('Your restaurant recommendation is Pho at Back of Urban Place in Gangnam').
suggest(rest_23) :- neighborhood(itaewon), vegeterian(not_serves), cuisine_type(turkish), 

In [49]:
KB = """
% Enter your KB below this line:

suggest(rest_8) :- neighborhood(gangnam), vegeterian(serves), cuisine_type(vietnamese), alcohol(serves), take_away(not_offers), coffee(not_serves), pescaterian(not_serves), open(w24/7), price_range(w7500_15000), write_py('Your restaurant recommendation is EMOI Pho at Gangnam station, Exit 3 in Gangnam').
suggest(rest_9) :- neighborhood(gangnam), vegeterian(serves), cuisine_type(vietnamese), alcohol(serves), take_away(not_offers), coffee(not_serves), pescaterian(not_serves), open(late), price_range(w7500_15000), write_py('Your restaurant recommendation is EMOI Pho at Sinsa-dong Garosu-Gil in Gangnam').
suggest(rest_12) :- neighborhood(gangnam), vegeterian(not_serves), cuisine_type(vietnamese), alcohol(not_serves), take_away(offers), coffee(not_serves), pescaterian(not_serves), open(early), price_range(w5000_7500), write_py('Your restaurant recommendation is Pho at Back of Urban Place in Gangnam').
suggest(rest_23) :- neighborhood(itaewon), vegeterian(not_serves), cuisine_type(turkish), alcohol(not_serves), take_away(offers), coffee(not_serves), pescaterian(not_serves), open(early), price_range(w5000_7500), write_py('Your restaurant recommendation is Sultan Turkish Kebab at 127-28 Itaewon 1(il)-dong in Itaewon').
suggest(rest_2) :- neighborhood(gangnam), vegeterian(not_serves), cuisine_type(taiwanese), alcohol(serves), take_away(not_offers), coffee(not_serves), pescaterian(serves), open(early), price_range(w15000_25000), write_py('Your restaurant recommendation is Din Tai Fung at 1317-31 Seocho-dong, Seocho-gu in Gangnam').
suggest(rest_4) :- neighborhood(gangnam), vegeterian(not_serves), cuisine_type(korean), alcohol(serves), take_away(not_offers), coffee(not_serves), pescaterian(not_serves), open(late), price_range(w7500_15000), write_py('Your restaurant recommendation is KBBQ at 810-1, Yeoksam-dong, Gangnam-gu, Seoul in Gangnam').
suggest(rest_6) :- neighborhood(gangnam), vegeterian(serves), cuisine_type(korean), alcohol(not_serves), take_away(not_offers), coffee(serves), pescaterian(not_serves), open(early), price_range(w7500_15000), write_py('Your restaurant recommendation is Food Court Gangnam Exit 3 at Gangam Station, Exit 3 in Gangnam').
suggest(rest_10) :- neighborhood(gangnam), vegeterian(not_serves), cuisine_type(korean), alcohol(not_serves), take_away(not_offers), coffee(not_serves), pescaterian(not_serves), open(early), price_range(w7500_15000), write_py('Your restaurant recommendation is Bulgogi at Gangnam station, Exit 5 in Gangnam').
suggest(rest_19) :- neighborhood(gangnam), vegeterian(not_serves), cuisine_type(korean), alcohol(serves), take_away(offers), coffee(not_serves), pescaterian(not_serves), open(early), price_range(w7500_15000), write_py('Your restaurant recommendation is Stir Fried Pork&Beef Place at 37°29\\'44.2"N 127°01\\'50.5"E in Gangnam').
suggest(rest_21) :- neighborhood(gangnam), vegeterian(not_serves), cuisine_type(korean), alcohol(serves), take_away(offers), coffee(not_serves), pescaterian(serves), open(late), price_range(w7500_15000), write_py('Your restaurant recommendation is Nollan Chicken 화난치킨 at 1327-15 Seocho-dong in Gangnam').
suggest(rest_22) :- neighborhood(gangnam), vegeterian(not_serves), cuisine_type(korean), alcohol(serves), take_away(not_offers), coffee(not_serves), pescaterian(not_serves), open(w24/7), price_range(w7500_15000), write_py('Your restaurant recommendation is Soup Place 24h at 37°29\\'39.4"N 127°01\\'41.2"E in Gangnam').
suggest(rest_25) :- neighborhood(gangnam), vegeterian(not_serves), cuisine_type(korean), alcohol(serves), take_away(not_offers), coffee(not_serves), pescaterian(not_serves), open(early), price_range(w7500_15000), write_py('Your restaurant recommendation is Lamb Soup/BBQ at 37°29\\'41.8"N 127°01\\'41.3"E in Gangnam').
suggest(rest_1) :- neighborhood(gangnam), vegeterian(serves), cuisine_type(korean), alcohol(not_serves), take_away(offers), coffee(not_serves), pescaterian(serves), open(early), price_range(w5000_7500), write_py('Your restaurant recommendation is Robot Kimbap at Gangnam station in Gangnam').
suggest(rest_3) :- neighborhood(gangnam), vegeterian(not_serves), cuisine_type(korean), alcohol(not_serves), take_away(offers), coffee(not_serves), pescaterian(not_serves), open(late), price_range(w5000_7500), write_py('Your restaurant recommendation is Meal Rice Dumplings at 1328-11 Seocho 2(i)-dong, Seocho-gu, Seoul in Gangnam').
suggest(rest_26) :- neighborhood(gangnam), vegeterian(serves), cuisine_type(korean), alcohol(serves), take_away(offers), coffee(not_serves), pescaterian(serves), open(w24/7), price_range(w5000_7500), write_py('Your restaurant recommendation is Gimbap Place We Have Deal With at Near UP in Gangnam').
suggest(rest_28) :- neighborhood(gangnam), vegeterian(serves), cuisine_type(korean), alcohol(serves), take_away(not_offers), coffee(not_serves), pescaterian(serves), open(late), price_range(w5000_7500), write_py('Your restaurant recommendation is Assembly line Udon place at Near 817 817 Yeoksam 1(il)-dong in Gangnam').
suggest(rest_0) :- neighborhood(gangnam), vegeterian(not_serves), cuisine_type(japanese), alcohol(serves), take_away(not_offers), coffee(not_serves), pescaterian(serves), open(early), price_range(w7500_15000), write_py('Your restaurant recommendation is Small Fish / Japanese restaurant at 137 Yeoksam-ro in Gangnam').
suggest(rest_15) :- neighborhood(gangnam), vegeterian(serves), cuisine_type(japanese), alcohol(serves), take_away(offers), coffee(not_serves), pescaterian(serves), open(early), price_range(w7500_15000), write_py('Your restaurant recommendation is Yutalo 雄太郎 at 1330-6 Seocho-dong in Gangnam').
suggest(rest_16) :- neighborhood(gangnam), vegeterian(serves), cuisine_type(japanese), alcohol(not_serves), take_away(offers), coffee(not_serves), pescaterian(serves), open(early), price_range(w7500_15000), write_py('Your restaurant recommendation is Kore Curry 고레카레 강남본점 at 1330-10 Seocho 2(i)-dong in Gangnam').
suggest(rest_27) :- neighborhood(gangnam), vegeterian(serves), cuisine_type(japanese), alcohol(not_serves), take_away(offers), coffee(not_serves), pescaterian(serves), open(early), price_range(w7500_15000), write_py('Your restaurant recommendation is Marugame Seimen　丸亀製麺 at Gangnam station exit 11 food street in Gangnam').
suggest(rest_11) :- neighborhood(gangnam), vegeterian(serves), cuisine_type(japanese), alcohol(not_serves), take_away(offers), coffee(not_serves), pescaterian(not_serves), open(w24/7), price_range(w5000_7500), write_py('Your restaurant recommendation is Japanese Donkatsu at Gangnam station, Exit 4 in Gangnam').
suggest(rest_13) :- neighborhood(gangnam), vegeterian(serves), cuisine_type(japanese), alcohol(serves), take_away(not_offers), coffee(not_serves), pescaterian(not_serves), open(late), price_range(w25000+), write_py('Your restaurant recommendation is Shabu Shabu at Urban Place B1 in Gangnam').
suggest(rest_5) :- neighborhood(gangnam), vegeterian(serves), cuisine_type(italian), alcohol(serves), take_away(not_offers), coffee(serves), pescaterian(serves), open(late), price_range(w15000_25000), write_py('Your restaurant recommendation is Sarubia at 542-3, Sinsa-dong, Gangnam-gu, Seoul in Gangnam').
suggest(rest_20) :- neighborhood(gangnam), vegeterian(serves), cuisine_type(chinese), alcohol(serves), take_away(offers), coffee(not_serves), pescaterian(serves), open(early), price_range(w7500_15000), write_py('Your restaurant recommendation is Ririxiang(?) 日日香 at 311 Gangnam-daero, Seocho 2(i)-dong in Gangnam').
suggest(rest_7) :- neighborhood(gangnam), vegeterian(serves), cuisine_type(breakfast), alcohol(serves), take_away(not_offers), coffee(serves), pescaterian(serves), open(late), price_range(w7500_15000), write_py('Your restaurant recommendation is Bosque at Gangnam-daero in Gangnam').
suggest(rest_14) :- neighborhood(gangnam), vegeterian(not_serves), cuisine_type(american), alcohol(serves), take_away(offers), coffee(serves), pescaterian(not_serves), open(early), price_range(w7500_15000), write_py('Your restaurant recommendation is KFC at 1330-8 Seocho-dong in Gangnam').
suggest(rest_17) :- neighborhood(gangnam), vegeterian(serves), cuisine_type(american), alcohol(not_serves), take_away(offers), coffee(serves), pescaterian(serves), open(w24/7), price_range(w7500_15000), write_py('Your restaurant recommendation is McDonald\\'s 맥도날드 서초뱅뱅점 at 1338-20 Seocho-dong in Gangnam').
suggest(rest_18) :- neighborhood(gangnam), vegeterian(not_serves), cuisine_type(american), alcohol(not_serves), take_away(offers), coffee(serves), pescaterian(serves), open(w24/7), price_range(w7500_15000), write_py('Your restaurant recommendation is Lotteria at 37°29\\'34.5"N 127°01\\'49.7"E in Gangnam').
suggest(rest_24) :- neighborhood(gangnam), vegeterian(serves), cuisine_type(american), alcohol(not_serves), take_away(offers), coffee(serves), pescaterian(serves), open(late), price_range(w15000_25000), write_py('Your restaurant recommendation is Shake Shack Gangnam at 452 Gangnam-daero in Gangnam').

neighborhood(X) :- check(neighborhood, X).
vegeterian(X) :- check(vegeterian, X).
cuisine_type(X) :- check(cuisine_type, X).
alcohol(X) :- check(alcohol, X).
take_away(X) :- check(take_away, X).
coffee(X) :- check(coffee, X).
pescaterian(X) :- check(pescaterian, X).
open(X) :- check(open, X).
price_range(X) :- check(price_range, X).

multivalued(neighborhood).
multivalued(cuisine_type).
multivalued(price_range).
multivalued(open).

% Care-checking implemented to allow the user to ignore attributes they don't care about:

check(A, X) :- not(check_if_care(A, X)).
check_if_care(A, X) :- care(A), not(ask(A, X)).
care(A) :- atom_concat(care_check_, A, X), ask(X, '').

% Asking clauses

ask(A, V):-
known(y, A, V), % succeed if true
!. % stop looking

ask(A, V):-
known(_, A, V), % fail if false
!, fail.

ask(A, V):-
not(multivalued(A)),
% write_py(A:not_multivalued),
known(y, A, V2),
V \== V2,
!, fail.

ask(A, V):-
read_py(A,V,Y), % get the answer
asserta(known(Y, A, V)), % remember it
Y == y. % succeed or fail
"""

with open("KB_A.pl", "w") as text_file:
    text_file.write(KB)

In [50]:
# The code here will ask the user for input based on the askables
# It will check if the answer is known first

from pyswip.prolog import Prolog
from pyswip.easy import *

prolog = Prolog() # Global handle to interpreter

retractall = Functor("retractall")
known = Functor("known",3)

# Define foreign functions for getting user input and writing to the screen
def write_py(X):
    print(str(X))
    sys.stdout.flush()
    return True


# This dictionary maps a variable name to a string template 
# of how to ask the user about it in a friendly manner
DEFAULT_QUESTION = '{A} is {V}? '
ATOM_TO_QUESTION_MAPPING = defaultdict(lambda: DEFAULT_QUESTION)
ATOM_TO_QUESTION_MAPPING.update({
    'neighborhood': 'Do you wish to eat in the {V} {A} ? ',
    'vegeterian': 'Do you need a place that {V} {A} friendly food? ',
    'pescaterian': 'SDo you need a place that {V} {A} friendly food? ',
    'cuisine_type': 'Do you wish to eat {V} food tonight? ',
    'alcohol': 'Do you need a place that {V} {A}? ',
    'coffee': 'Do you need a place that {V} {A}? ',
    'take_away': 'Do you need a place that {V} take-away? ',
    'open': 'Would you like a place that is {A} {V}? ',
    'price_range': 'Are you okay with a price range of {V} per person? '
})

NEGATION_PREFIX = 'not_'
NEGATION_REPLACEMENT = 'does not '

CARE_CHECK_PREFIX = 'care_check_'
DEFAULT_CARE_QUESTION = 'Do you care about {A}? '
CARE_CHECK_QUESTION_MAPPING = defaultdict(lambda: DEFAULT_CARE_QUESTION)
CARE_CHECK_QUESTION_MAPPING.update({
    'neighborhood': 'Do you care which {A} you eat in? ',
    'vegeterian': 'Do you care if the restaurant is {A} friendly? ',
    'pescaterian': 'Do you care if the restaurant is {A} friendly? ',
    'cuisine_type': 'Do you care about which cuisine you eat tonight? ',
    'alcohol': 'Do you care if the restaurant serves {A}? ',
    'coffee': 'Do you care if the restaurant serves {A}? ',
    'take_away': 'Do you care if the restaurant offers take-away? ',
    'open': 'Do you care if the restaurant is open late or 24/7? ',
    'price_range': 'Do you care about expected price range of the restaurant? '
})

YES_VALUES = ('yes', 'y', 'true', 'sure', 'okay', 'definitely', 'cool')

def read_py(A,V,Y):
    A_value = A.get_value() if type(A) == Atom else str(A)
    V_value = V.get_value() if type(V) == Atom else str(V)
    
    if A_value.startswith(CARE_CHECK_PREFIX):
        A_value = A_value[len(CARE_CHECK_PREFIX):]
        question = CARE_CHECK_QUESTION_MAPPING[A_value]
   
    else:
        question = ATOM_TO_QUESTION_MAPPING[A_value]
        if V_value.startswith(NEGATION_PREFIX):
            V_value = NEGATION_REPLACEMENT + V_value[len(NEGATION_PREFIX):-1]
    
    if A_value == 'price_range':
        V_value = '₩' + V_value[1:].replace('_', '-')
        
    user_input = raw_input(question.format(A=A_value, V=V_value))
    
    if user_input.lower() in YES_VALUES:
        user_input = 'y'
    
    Y.unify(user_input)
    return True


write_py.arity = 1
read_py.arity = 3

registerForeign(read_py)
registerForeign(write_py)

prolog.consult("KB_A.pl") # open the KB
call(retractall(known))
found = False
max_results = None

print('How many reccomendations would you like to see (if enough restaurants match your specifications)?')
print('Leave empty for 1, use 0 for "as many as possible"')
while max_results is None:
    user_value = raw_input()
    if user_value == '':
        user_value = '1'
        
    if user_value.isdigit():
        max_results = int(user_value)
    else:
        print('Please enter an integer number (or empty)')

if 0 == max_results:
    max_results = 1000
        
for soln in prolog.query("suggest(X).", maxresult=max_results):
    found = True
    
if not found:
    print('Sorry, no acceptable restaurants found, please try again.')

How many reccomendations would you like to see (if enough restaurants match your specifications)?
Leave empty for 1, use 0 for "as many as possible"
0
Do you care which neighborhood you eat in? n
Do you care if the restaurant is vegeterian friendly? n
Do you care about which cuisine you eat tonight? y
Do you wish to eat vietnamese food tonight? n
Do you wish to eat turkish food tonight? n
Do you wish to eat taiwanese food tonight? n
Do you wish to eat korean food tonight? y
Do you care if the restaurant serves alcohol? n
Do you care if the restaurant offers take-away? y
Do you need a place that does not offer take-away? y
Do you care if the restaurant serves coffee? n
Do you care if the restaurant is pescaterian friendly? n
Do you care if the restaurant is open late or 24/7? n
Do you care about expected price range of the restaurant? n
Your restaurant recommendation is KBBQ at 810-1, Yeoksam-dong, Gangnam-gu, Seoul in Gangnam
Your restaurant recommendation is Food Court Gangnam Exit 3 